# **Importing Important Libraries**

In [2]:
import pandas as pd
!pip install pymysql
import pymysql
from sqlalchemy import create_engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00


# **Importing the files and making a dataframe for each worksheet**

In [ ]:
# the maximum size of a python list on a 32 bit system is 536,870,912 elements

files = ["C:\\Users\\Aradhya\\Downloads\\MohanUW_Demo.xlsx", "C:\\Users\\Aradhya\\Downloads\\rohanUW.xlsx",
         "C:\\Users\\Aradhya\\Downloads\\SohanUW.xlsx"]
dfs_borrowers = []
n1 = len(files)
for i in range(n1):
    df = pd.read_excel(files[i])
    df1 = df.T.reset_index(drop = True)
    df1.columns = df1.iloc[0]
    df1 = df1.iloc[1:,:]
    dfs_borrowers.append(df1)
borrower_df = pd.concat(dfs_borrowers, ignore_index = True)
borrower_df

,Name,CIBIL,Address
0,Mohan Kumar,640,"C/340, Gomti Nagar, Lucknow, U.P."
1,Rohan Kumar,670,Agra
2,Sohan Kumar,640,"Sector-16, Noida"


In [ ]:
files = ["C:\\Users\\Aradhya\\Downloads\\MohanUW_Demo.xlsx", "C:\\Users\\Aradhya\\Downloads\\rohanUW.xlsx",
         "C:\\Users\\Aradhya\\Downloads\\SohanUW.xlsx"]
dfs_details = []
n = len(files)
for i in range(n):
    df = pd.read_excel(files[i], sheet_name = 'Bank-statement')
    name = dfs_borrowers[i]['Name'].item()
    address = dfs_borrowers[i]['Address'].item()
    df['Name'] = name
    df['Address'] =  address
    dfs_details.append(df)
bank_df = pd.concat(dfs_details, ignore_index = True)
bank_df

,Date,Narration,Ref #,Dr,Cr,Balance,Name,Address
0,2023-04-10 00:00:00,RTGS. Ritesh..,45567677,5000.0,NaN,45000,Mohan Kumar,"C/340, Gomti Nagar, Lucknow, U.P."
1,2023-04-10 00:00:00,EMI.,56566788,7000.0,NaN,38000,Mohan Kumar,"C/340, Gomti Nagar, Lucknow, U.P."
2,2023-04-17 00:00:00,NEFT/….,56576778,NaN,40000.0,78000,Mohan Kumar,"C/340, Gomti Nagar, Lucknow, U.P."
3,2023-04-27 00:00:00,Interest credited,54657887,NaN,450.0,78450,Mohan Kumar,"C/340, Gomti Nagar, Lucknow, U.P."
4,2023-05-06 00:00:00,UPI…,45788999,1000.0,NaN,77450,Mohan Kumar,"C/340, Gomti Nagar, Lucknow, U.P."
5,31-04-2023,IMPS,45567673,5000.0,NaN,45000,Rohan Kumar,Agra
6,2023-05-01 00:00:00,Interest credited,55433488,7000.0,NaN,38000,Rohan Kumar,Agra
7,2023-05-08 00:00:00,Interest credited,56560778,NaN,40000.0,78000,Rohan Kumar,Agra
8,2023-05-18 00:00:00,EMI,54557887,NaN,450.0,78450,Rohan Kumar,Agra
9,2023-05-27 00:00:00,RTGS,45788009,1000.0,NaN,77450,Rohan Kumar,Agra


# **Linking with Database**

In [3]:
MYSQL_HOST = 'localhost'
MYSQL_USER = 'root'
MYSQL_PASSWORD = 'password%40123'
MYSQL_DB = 'sample_db'

engine = create_engine(f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}/{MYSQL_DB}')

In [ ]:
def update_mysql_table(df, table_name):
    existing_data = pd.read_sql_table(table_name, engine)
    new_rows = df[~df.isin(existing_data.to_dict('list')).all(1)]
    if not new_rows.empty:
        new_rows.to_sql(table_name, engine, if_exists='append', index=False)

In [ ]:
borrower_table = 'Borrowers'
bank_table = 'BankStatements'
update_mysql_table(borrower_df, borrower_table)
update_mysql_table(bank_df, bank_table)